In [108]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [109]:
df_item = pd.read_csv('../../Dataset/Local Attraction/data_encoded (4).csv')
df_item.head()

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Time_Minutes,Coordinate,Lat,...,_1,Rating_Count,Alam,Hiburan,Kuliner,Olahraga,Rekreasi,Religius,Sejarah Edukasi,Seni Budaya
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Sejarah Edukasi,Jakarta,20000,46,15.0,"{'lat': -6.1753924, 'lng': 106.8271528}",-61753924,...,1,18,0,0,0,0,0,0,1,0
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...","Sejarah Edukasi,Seni Budaya,Kuliner",Jakarta,0,46,90.0,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-61376448,...,2,25,0,0,1,0,0,0,1,1
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,"Rekreasi,Hiburan",Jakarta,270000,46,360.0,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-61253124,...,3,19,0,1,0,0,1,0,0,0
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,"Sejarah Edukasi,Seni Budaya,Rekreasi",Jakarta,10000,45,NaN,"{'lat': -6.302445899999999, 'lng': 106.8951559}",-63024459,...,4,21,0,0,0,0,1,0,1,1
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,"Rekreasi,Olahraga",Jakarta,94000,45,60.0,"{'lat': -6.12419, 'lng': 106.839134}",-612419,...,5,24,0,0,0,1,1,0,0,0


In [110]:
df_item.columns

Index(['Place_Id', 'Place_Name', 'Description', 'Category', 'City', 'Price',
       'Rating', 'Time_Minutes', 'Coordinate', 'Lat', 'Long', 'Column1', '_1',
       'Rating_Count', 'Alam', 'Hiburan', 'Kuliner', 'Olahraga', 'Rekreasi',
       'Religius', 'Sejarah Edukasi', 'Seni Budaya'],
      dtype='object')

In [111]:
drop_columns = df_item.columns[2:14]
df_item = df_item.drop(drop_columns, axis=1)
df_item.head()

,Place_Id,Place_Name,Alam,Hiburan,Kuliner,Olahraga,Rekreasi,Religius,Sejarah Edukasi,Seni Budaya
0,1,Monumen Nasional,0,0,0,0,0,0,1,0
1,2,Kota Tua,0,0,1,0,0,0,1,1
2,3,Dunia Fantasi,0,1,0,0,1,0,0,0
3,4,Taman Mini Indonesia Indah (TMII),0,0,0,0,1,0,1,1
4,5,Atlantis Water Adventure,0,0,0,1,1,0,0,0


In [112]:
df_user = pd.read_csv('../../Dataset/User Rating/user_preference.csv')
df_user.head()

,User,Alam,Hiburan,Kuliner,Olahraga,Rekreasi,Religius,Sejarah Edukasi,Seni Budaya
0,User 1,0.363,0.260109,0.213392,0.0,0.226027,0.0,0.189003,0.587863
1,User 2,0.363,0.261202,0.213392,0.0,0.248097,0.0,0.189003,0.587863
2,User 3,0.363,0.260109,0.213392,0.0,0.246575,0.0,0.189003,0.587863
3,User 4,0.363,0.260109,0.213392,0.0,0.224505,0.0,0.189003,0.587863
4,User 5,0.363,0.260109,0.213392,0.0,0.246575,0.0,0.189003,0.587863


In [113]:
df_user.columns = df_user.iloc[0]
drop_columns = df_user.columns[0]
df_user = df_user.drop(drop_columns, axis=1)
df_user = df_user.iloc[:df_item.shape[0]]

df_item.columns = df_item.iloc[0]
drop_columns = df_item.columns[:2]
df_item = df_item.drop(drop_columns, axis=1)

In [114]:
user_train, user_test = train_test_split(df_user, test_size=0.2, random_state=42)
item_train, item_test = train_test_split(df_item, test_size=0.2, random_state=42)
user_train.shape, user_test.shape, item_train.shape, item_test.shape

feature_user = user_train.shape[1]
feature_item = item_train.shape[1]

In [115]:
user_model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32, activation='linear'),
])
item_model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32, activation='linear'),
])

input_user = tf.keras.layers.Input(shape=(feature_user))  
input_item = tf.keras.layers.Input(shape=(feature_item))

vector_user = tf.linalg.l2_normalize(user_model(input_user), axis=1)
vector_item = tf.linalg.l2_normalize(item_model(input_item), axis=1)

output = tf.keras.layers.Dot(axes=1)([vector_user, vector_item])
model = tf.keras.Model(inputs=[input_user, input_item], outputs=output)
model.summary()
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 8)]          0           []                               
                                                                                                  
 input_16 (InputLayer)          [(None, 7)]          0           []                               
                                                                                                  
 sequential_14 (Sequential)     (None, 32)           39328       ['input_15[0][0]']               
                                                                                                  
 sequential_15 (Sequential)     (None, 32)           39072       ['input_16[0][0]']               
                                                                                            

In [122]:
model.fit([user_train, item_train],tf.ones_like(user_train), epochs=100, batch_size=32)

Epoch 1/100
 1/11 [=>............................] - ETA: 0s - loss: 2.3836e-06 - mse: 2.3836e-06

11/11 [==============================] - 0s 22ms/step - loss: 2.7052e-06 - mse: 2.7052e-06
Epoch 2/100
11/11 [==============================] - 0s 9ms/step - loss: 2.6608e-06 - mse: 2.6608e-06
Epoch 3/100
11/11 [==============================] - 0s 15ms/step - loss: 2.6169e-06 - mse: 2.6169e-06
Epoch 4/100
11/11 [==============================] - 0s 11ms/step - loss: 2.5731e-06 - mse: 2.5731e-06
Epoch 5/100
11/11 [==============================] - 0s 16ms/step - loss: 2.5343e-06 - mse: 2.5343e-06
Epoch 6/100
11/11 [==============================] - 0s 16ms/step - loss: 2.4872e-06 - mse: 2.4872e-06
Epoch 7/100
11/11 [==============================] - 0s 15ms/step - loss: 2.4492e-06 - mse: 2.4492e-06
Epoch 8/100
11/11 [==============================] - 0s 9ms/step - loss: 2.4111e-06 - mse: 2.4111e-06
Epoch 9/100
11/11 [==============================] - 0s 8ms/step - loss: 2.3709e-06 - mse: 2.3709e-06
Epoch 10/100
11/11 [==============================] - 0s 7ms/step - loss: 2.3354e-06 - m

In [121]:
model.evaluate([user_test, item_test], tf.ones_like(user_test))

3/3 [==============================] - 0s 13ms/step - loss: 3.9349e-06 - mse: 3.9349e-06


[3.934910182579188e-06, 3.934910182579188e-06]